# Mixed Likelihood GPLVM

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import time

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from IPython import display
%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
try:
    import tfgp
    from tfgp.util import data
    from tfgp.model import MLGPLVM
    print(f"Succesfully imported package: {tfgp.__file__}")
except ModuleNotFoundError:
    import sys
    import os
    tfgpdir = os.path.abspath("../..")
    sys.path.append(tfgpdir)
    import tfgp
    from tfgp.util import data
    from tfgp.model import MLGPLVM
    print(f"Package not found, adding project root to python path and import: {tfgp.__file__}")

## Generate data

In [ ]:
#np.random.seed(1)

In [ ]:
num_data = 50
latent_dim = 2
output_dim = 5
num_classes = 4
y, likelihoods, labels = data.make_gaussian_blobs(num_data, output_dim, num_classes)
x = tfgp.util.pca_reduce(y, latent_dim)

In [ ]:
plt.scatter(*x.T, c=labels, cmap="Paired")

## Create model

In [ ]:
kernel = tfgp.kernel.ARDRBF(variance=0.5, gamma=0.5, xdim=2, name="kernel")
m = MLGPLVM(y, latent_dim, num_inducing=20, kernel=kernel, likelihoods=likelihoods)

## Build graph

In [ ]:
loss = tf.losses.get_total_loss()
learning_rate = 1e-3
with tf.name_scope("train"):
    trainable_vars = tf.trainable_variables()
    optimizer = tf.train.RMSPropOptimizer(learning_rate, name="RMSProp")
    train_all = optimizer.minimize(loss, 
                                   var_list=tf.trainable_variables(),
                                   global_step=tf.train.create_global_step(),
                                   name="train")

with tf.name_scope("summary"):
    m.create_summaries()
    tf.summary.scalar("total_loss", loss, family="Loss")
    for reg_loss in tf.losses.get_regularization_losses():
        tf.summary.scalar(f"{reg_loss.name}", reg_loss, family="Loss")
    merged_summary = tf.summary.merge_all()
init = tf.global_variables_initializer()

## Run optimisation

In [ ]:
def plot(x: np.ndarray, *, z: np.ndarray = None, y_pred: np.ndarray = None, loss) -> None:
    ax1.scatter(*x.T, c=labels, cmap="Paired", edgecolors='k')
    if z is not None:
        ax1.scatter(*z.T, c="k", marker="x")
    if y_pred is not None:
        ax1.plot(*x.T)
        ax3.plot(y - y_pred)
    ax1.set_title(f"Step {i}")
    ax2.set_title(f"Loss: {train_loss}")
    ax2.semilogy(*np.array(loss).T)
    display.display(f)
    display.clear_output(wait=True)

In [ ]:
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
log_dir = f"../../log/mlgplvm/{time.strftime('%Y%m%d%H%M%S')}"
loss_list = []
n_iter = 100000
n_print = 500
try:
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
    sess.run(init)
    for i in range(n_iter):
        sess.run(train_all)
        if i % n_print == 0:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            train_loss, summary = sess.run([loss, merged_summary], options=run_options, run_metadata=run_metadata)
            summary_writer.add_run_metadata(run_metadata, f"step_{i}", global_step=i)
            summary_writer.add_summary(summary, i)
            x_mean = m.qx_mean.eval().T
            z = m.z.eval()
            y_mean, _ = sess.run(m.predict(x))
            loss_list.append([i, train_loss])
            plot(x_mean, z=z, loss=loss_list)
            ax1.cla()
            ax2.cla()
            ax3.cla()
except KeyboardInterrupt:
    pass
finally:
    x_mean = m.qx_mean.eval().T
    z = m.z.eval()
    y_mean, _ = sess.run(m.predict(x))
    loss_list.append([i, loss.eval()])
    plot(x_mean, z=z, loss=loss_list)
